# Capturar movimento do mouse
Nesta seção, vamos criar uma interface utilizando a biblioteca **pygame**, contendo três alvos. O usuário deverá dar um **duplo clique** no primeiro alvo para iniciar a captura do mouse. Em seguida, deve movimentar o mouse até o alvo azul e realizar um **clique simples**; depois, guiar o mouse até o alvo vermelho e dar um **duplo clique** para encerrar a captura.

In [73]:
import pygame
import time
import pandas as pd
from math import hypot

## Parâmetros e variáveis

In [75]:
# EXECUTAR: parâmetros

# Parâmetros de captura
DOUBLE_CLICK_MAX_INTERVAL = 0.3  # segundos
POINT_RADIUS = 20

# Parâmetros dos alvos (offset)
x_offset_inicio = -150
y_offset_inicio = 150

x_offset_intermediario = 0
y_offset_intermediario = -120

x_offset_fim = 0
y_offset_fim = 50

# Lista de eventos e outras variáveis
eventos = []
ultimo_click = {}
click_pendente = {}
capturando = False

## Funções

In [76]:
def registrar_evento(x, y, event_type, button, click_count):
    eventos.append({
        'timestamp': time.time(),
        'x': x,
        'y': y,
        'event_type': event_type,
        'button': button,
        'click_count': click_count
    })

In [77]:
def detect_click(x, y, button):
    global ultimo_click, click_pendente
    now = time.time()
    key = str(button)

    # Se já tem clique pendente
    if key in click_pendente:
        prev = click_pendente.pop(key)
        dt = now - prev['timestamp']
        dist = hypot(x - prev['x'], y - prev['y'])
        if dt <= DOUBLE_CLICK_MAX_INTERVAL and dist <= POINT_RADIUS:
            # é duplo clique
            registrar_evento(x, y, 'double_click', button, 2)
            return 'double_click', 2
        else:
            # registra o clique pendente como simples
            registrar_evento(prev['x'], prev['y'], 'click', button, 1)

    # guarda este clique como pendente
    click_pendente[key] = {'timestamp': now, 'x': x, 'y': y}
    return 'click_pending', 0


In [78]:
def processar_cliques_pendentes():
    now = time.time()
    keys_para_remover = []
    for key, clique in click_pendente.items():
        if now - clique['timestamp'] > DOUBLE_CLICK_MAX_INTERVAL:
            # Registrando clique simples
            registrar_evento(clique['x'], clique['y'], 'click', key, 1)
            keys_para_remover.append(key)
    # Remove cliques processados
    for key in keys_para_remover:
        click_pendente.pop(key)

In [79]:
def inicializar_pygame():
    pygame.init()
    info = pygame.display.Info()
    global WIDTH, HEIGHT
    WIDTH, HEIGHT = info.current_w // 2, info.current_h // 2
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Simulação de Trajetória do Mouse")

    # Definir pontos alvos (x, y)
    ponto_inicio = (WIDTH // 4, HEIGHT // 2)
    ponto_intermediario = (WIDTH // 2, HEIGHT // 2)
    ponto_fim = (3 * WIDTH // 4, HEIGHT // 2)

    return screen, ponto_inicio, ponto_intermediario, ponto_fim


## Excução

In [80]:
#EXECUTAR: capturar movimento do mouse

# Limpar variáveis
eventos = []
ultimo_click = {}

# Inicializar pygame
pygame.init()
info = pygame.display.Info()
WIDTH, HEIGHT = info.current_w // 2, info.current_h // 2
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simulação de Trajetória do Mouse")

# Definir pontos alvos (x, y)
ponto_inicio = ((WIDTH // 4) + x_offset_inicio, (HEIGHT // 2) + y_offset_inicio)
ponto_intermediario = ((WIDTH // 2) + x_offset_intermediario, (HEIGHT // 2) + y_offset_intermediario)
ponto_fim = ((3 * WIDTH // 4) + x_offset_fim, (HEIGHT // 2) + y_offset_fim)

# Variável de controle
rodando = True

# Loop principal
while rodando:
    screen.fill((255, 255, 255))

    # Desenhar pontos
    pygame.draw.circle(screen, (0, 255, 0), ponto_inicio, POINT_RADIUS)  # verde
    pygame.draw.circle(screen, (0, 0, 255), ponto_intermediario, POINT_RADIUS)  # azul
    pygame.draw.circle(screen, (255, 0, 0), ponto_fim, POINT_RADIUS)  # vermelho
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            rodando = False
        elif event.type == pygame.MOUSEBUTTONUP:
            x, y = event.pos
            button = event.button
            click_type, click_count = detect_click(x, y, button)

            if not capturando and click_type == 'double_click' and hypot(x - ponto_inicio[0], y - ponto_inicio[1]) <= POINT_RADIUS:
                capturando = True  # início da captura
                print("Captura iniciada")

            if capturando and click_type == 'double_click' and hypot(x - ponto_fim[0], y - ponto_fim[1]) <= POINT_RADIUS:
                capturando = False
                rodando = False
                print("Captura encerrada")

        elif event.type == pygame.MOUSEMOTION and capturando:
            x, y = event.pos
            registrar_evento(x, y, 'move', 'mouse', 0)

    # Checar cliques pendentes
    processar_cliques_pendentes()

pygame.quit()


Captura iniciada
Captura encerrada


## Resultado

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

In [82]:
%matplotlib qt

In [83]:
# Transformar lista em DataFrame
df = pd.DataFrame(eventos)
print(f"{len(eventos)} eventos registrados.")

177 eventos registrados.


In [84]:
display(df[df['click_count']!=0])

,timestamp,x,y,event_type,button,click_count
0,1.758214e+09,83,418,double_click,1,2
113,1.758214e+09,474,149,click,1,1
176,1.758214e+09,719,327,double_click,1,2


In [85]:
# EXECUTAR: plotar

# Configurações da janela (mesmo tamanho que o pygame)
WIDTH, HEIGHT = info.current_w // 2, info.current_h // 2

# Criar figura
fig, ax = plt.subplots(figsize=(WIDTH/100, HEIGHT/100), dpi=100)
ax.set_xlim(0, WIDTH)
ax.set_ylim(0, HEIGHT)
ax.invert_yaxis()  # coordenadas como no pygame (0,0 no topo)

# Traçar trajetória
movimentos = df[df['event_type'] == 'move']
ax.plot(movimentos['x'], movimentos['y'], color='black', linewidth=1)

# Marcar cliques
cliques = df[df['event_type'].isin(['click', 'double_click'])]
for _, row in cliques.iterrows():
    x, y = row['x'], row['y']
    if row['event_type'] == 'click':
        ax.scatter(x, y, color='blue', s=50, label='Clique')
        ax.text(x + 5, y + 5, '1x', fontsize=8)
    else:  # double_click
        ax.scatter(x, y, color='red', s=70, label='Duplo clique')
        ax.text(x + 5, y + 5, '2x', fontsize=8)

# Evitar múltiplas legendas repetidas
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())

plt.title("Trajetória do Mouse (simulação)")
plt.show()  # abre em janela separada, não trava o notebook


# Simular tremores
Nesta seção, utilizaremos os dados capturados anteriormente e adicionaremos **ruído aleatório** às posições, com o objetivo de **simular tremores**.

In [93]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

## Funções

In [87]:
def gerar_ruido_aleatorio(df_mov, sigma_min=2, sigma_max=15):
    """
    Adiciona ruído aleatório nas coordenadas x e y, com intensidade variável
    de forma aleatória ao longo da trajetória.

    df_mov: DataFrame de movimentos
    sigma_min: desvio padrão mínimo
    sigma_max: desvio padrão máximo
    """
    n = len(df_mov)
    if n == 0:
        return df_mov

    # Intensidade do ruído sorteada aleatoriamente para cada ponto
    np.random.seed(42)  # reprodutibilidade
    sigma_x = np.random.uniform(sigma_min, sigma_max, n)
    sigma_y = np.random.uniform(sigma_min, sigma_max, n)

    df_mov['x_noisy'] = df_mov['x'] + np.random.normal(0, sigma_x)
    df_mov['y_noisy'] = df_mov['y'] + np.random.normal(0, sigma_y)
    return df_mov


## Executar

In [88]:
# Filtrar apenas movimentos
df_mov = df[df['event_type'] == 'move'].copy()

# Adicionar ruído variável
df_mov = gerar_ruido_aleatorio(df_mov, sigma_min=2, sigma_max=30)

In [89]:
df_cliques = df[df['event_type'].isin(['click', 'double_click'])].copy()
df_cliques['x_noisy'] = df_cliques['x']
df_cliques['y_noisy'] = df_cliques['y']

# Novo DataFrame com movimentos ruidosos + cliques originais
df_noisy = pd.concat([df_mov, df_cliques], ignore_index=True).sort_values('timestamp')

## Resultado

In [90]:
plt.figure(figsize=(10, 6))
plt.plot(df_noisy['x_noisy'], df_noisy['y_noisy'], color='black', linewidth=1)

# Marcar cliques
for _, row in df_noisy[df_noisy['event_type'].isin(['click', 'double_click'])].iterrows():
    color = 'blue' if row['event_type'] == 'click' else 'red'
    plt.scatter(row['x_noisy'], row['y_noisy'], color=color, s=50)

plt.gca().invert_yaxis()
plt.title("Trajetória com tremor variável simulado")
plt.show()


## Comparar

In [ ]:
# Criar figura
fig, ax = plt.subplots(figsize=(WIDTH/100, HEIGHT/100), dpi=100)
ax.set_xlim(0, WIDTH)
ax.set_ylim(0, HEIGHT)
ax.invert_yaxis()  # coordenadas como no pygame (0,0 no topo)

# Trajetória original
ax.plot(df[df['event_type'] == 'move']['x'],
        df[df['event_type'] == 'move']['y'],
        color='black', linewidth=1, label='Trajetória Original')

# Trajetória com ruído
ax.plot(df_noisy[df_noisy['event_type'] == 'move']['x_noisy'],
        df_noisy[df_noisy['event_type'] == 'move']['y_noisy'],
        color='orange', linewidth=1, label='Trajetória com Tremor')

# Marcar cliques
for df_plot, label_suffix in zip([df, df_noisy], ['Orig', 'Ruído']):
    for _, row in df_plot[df_plot['event_type'].isin(['click', 'double_click'])].iterrows():
        color = 'blue' if row['event_type'] == 'click' else 'red'
        ax.scatter(row['x'], row['y'], color=color, s=50)

# Legenda dos cliques
clique_patch = Line2D([0], [0], marker='o', color='w', label='Clique Único',
                       markerfacecolor='blue', markersize=8)
double_patch = Line2D([0], [0], marker='o', color='w', label='Duplo Clique',
                       markerfacecolor='red', markersize=8)
ax.legend(handles=[ax.get_lines()[0], ax.get_lines()[1], clique_patch, double_patch])

ax.set_title("Comparação: Trajetória Original x Trajetória com Tremor")
ax.set_xlabel("X")
ax.set_ylabel("Y")
plt.show()